### Expressive Quantitative Trait Loci (EQTL) Analysis

In this notebook we will show how to run a simple EQTL analyisis using the `cellink` package.

First we import the necessary libraries.

In [1]:
import logging
import warnings

import anndata as ad
import scanpy as sc
import numpy as np
import anndata as an
import pandas as pd

from tqdm.notebook import tqdm
from anndata.utils import asarray
from pathlib import Path
from statsmodels.stats.multitest import fdrcorrection

from cellink.io import read_sgkit_zarr
from cellink.tl import run_eqtl_on_single_gene

warnings.filterwarnings('ignore')

logger = logging.getLogger(__name__)

EQTL analyis requires reasoning over each cell type and each chromosome. In this tutorial we will be working on the chromosome 22 and the CD4 NC cell type.

In [2]:
DEBUG = False
TARGET_CELL_TYPE = "CD4 NC"
TARGET_CHROMOSOME = "22"
THRESHOLD = 0.05

Defining the data paths (should we do this differently when we publish the tutorial?)

In [3]:
## paths
DATA = Path("/home/lollo/Work/hackathon/data/Yazar_OneK1K")
#DATA = Path("/Users/jan.engelmann/projects/sc-eqtl/data")

vcf_file_path = DATA / "OneK1K_imputation_post_qc_r2_08/filter_vcf_r08/chr22.dose.filtered.R2_0.8.vcf.gz"

zarr_path = vcf_file_path.parent.parent / "filter_zarr_r08"
zarr_path.mkdir(exist_ok=True)

icf_file_path = zarr_path / vcf_file_path.with_suffix(".icf").name
zarr_file_path = (zarr_path / vcf_file_path.stem).with_suffix(".vcz")

if DEBUG:
    scdata_path = DATA / "debug_OneK1K_cohort_gene_expression_matrix_14_celltypes.h5ad"
else:
    scdata_path = DATA / "OneK1K_cohort_gene_expression_matrix_14_celltypes.h5ad.gz"

print(zarr_file_path, scdata_path)

/home/lollo/Work/hackathon/data/Yazar_OneK1K/OneK1K_imputation_post_qc_r2_08/filter_zarr_r08/chr22.dose.filtered.R2_0.8.vcz /home/lollo/Work/hackathon/data/Yazar_OneK1K/OneK1K_cohort_gene_expression_matrix_14_celltypes.h5ad.gz


Now we can read the single cell data by using the `anndata.read_h5ad` function.
Since we need to reason over each single cell type, we need to subset the data properly before proceeding with the rest of the pipeline.

In [4]:
## reading single cell data
scdata = ad.read_h5ad(scdata_path)
## filtering by the target cell type
scdata = scdata[scdata.obs.cell_label == TARGET_CELL_TYPE]
scdata

View of AnnData object with n_obs × n_vars = 463528 × 32738
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'pool', 'individual', 'percent.mt', 'latent', 'nCount_SCT', 'nFeature_SCT', 'cell_type', 'cell_label', 'sex', 'age'
    var: 'GeneSymbol', 'features'

We read the genetic data from the `zarr` file using the `cellink.tl.read_sgkit_data` API.

In [5]:
gdata = read_sgkit_zarr(zarr_file_path)
gdata.obs = gdata.obs.set_index("id")
gdata

AnnData object with n_obs × n_vars = 1034 × 143083
    var: 'chrom', 'pos', 'a0', 'a1', 'AF', 'ER2', 'maf', 'R2', 'contig', 'id', 'id_mask', 'quality'
    varm: 'filter'

In order to proceed with the analysis, we need to extend the single cell data with the biomart annotations.

In [6]:
## annotating the single cell data
annot = (
    sc.queries.biomart_annotations(
        "hsapiens",
        ["ensembl_gene_id", "start_position", "end_position", "chromosome_name"],
    )
    .set_index("ensembl_gene_id")
    .drop_duplicates()
)

scdata = scdata[:, scdata.var.index.isin(annot.index)]
scdata.var["chrom"] = annot.loc[scdata.var.index, "chromosome_name"].values
scdata.var["start"] = annot.loc[scdata.var.index, "start_position"].values
scdata.var["end"] = annot.loc[scdata.var.index, "end_position"].values

We can now normalize the counts and log-transform them

In [7]:
sc.pp.normalize_total(scdata)
sc.pp.log1p(scdata)
sc.pp.normalize_total(scdata)

Since we have the genetic data associated with chromosome 22 we need to subset the single cell data to contain only the genes that are associated to such chromosome, given the biomart annotation.

In [8]:
scdata = scdata[:, scdata.var.chrom == TARGET_CHROMOSOME]

Since each observation in the genetic data is a donor, we need to pseudo-bulk the single cell data to have a representation at the same level.

In [9]:
## aggregating the data
pbdata = sc.get.aggregate(scdata, "individual", "mean")
gdata = gdata[pbdata.obs.index]
pbdata.X = pbdata.layers["mean"]
pbdata

AnnData object with n_obs × n_vars = 981 × 666
    obs: 'individual'
    var: 'GeneSymbol', 'features', 'chrom', 'start', 'end'
    layers: 'mean'

We need to perform some sanity check to make sure that the observations match across the two data sources (pseudo-bulked single cell and genetic data)

In [10]:
## sanity check (we have all the individuals from both data sources)
assert (pbdata.obs.index == gdata.obs.index).all()

We will also filter out the genes that are expressed in less than 10 cells 

In [11]:
## first we need to filter out genes that are expressed in less than ten individuals
sc.pp.filter_genes(pbdata, min_cells = 10)

We can now run our EQTL test for each of the genes that are associated to the 22 chromosome

In [12]:
## retrieving the genes associated to chromosome 22
genes_chrom_22 = pbdata[:, pbdata.var["chrom"] == TARGET_CHROMOSOME].var.index.values
## running the eqtl test
cis_window = 1_000_000
results = []
## defining the iterator
iterator = tqdm(range(len(genes_chrom_22)))
for target_gene in genes_chrom_22:
    eqtl_results = run_eqtl_on_single_gene(pbdata, gdata, target_gene, cis_window)
    results.append(eqtl_results)
    iterator.update()

  0%|          | 0/494 [00:00<?, ?it/s]

To make more sense of the results, we need to consider the Bonferroni adjusted p-value along with the q-value computed by using the Benjamini-Hochberg score across the test.

In [16]:
## constructing output DataFrame
eqtl_results_df = pd.DataFrame(results)
eqtl_results_df["pv_reject"] = eqtl_results_df["min_pv"] < THRESHOLD
eqtl_results_df["bf_pv"] = np.clip(eqtl_results_df["min_pv"]*eqtl_results_df["no_tested_variants"], 0, 1)
eqtl_results_df["bf_pv_reject"] = eqtl_results_df["bf_pv"] < THRESHOLD
eqtl_results_df["q_val"] = fdrcorrection(eqtl_results_df["bf_pv"].values)[1]
eqtl_results_df["q_val_reject"] = eqtl_results_df["q_val"] < THRESHOLD

Once we have terminated our analysis, we can save the resulting `DataFrame` to disk

In [17]:
## saving the resulting dataframe
eqtl_results_df.to_csv(f"/home/lollo/Work/hackathon/dump/eqtl_{TARGET_CELL_TYPE}.csv")

In [18]:
eqtl_results_df

,terget_gene,no_tested_variants,min_pv,min_pv_variant,pv_reject,bf_pv,bf_pv_reject,q_val,q_val_reject
0,ENSG00000100181,1451,4.729347e-09,22_17274624_G_A,True,6.862283e-06,True,1.486828e-05,True
1,ENSG00000237438,3031,1.707060e-04,22_17529108_C_T,True,5.174098e-01,False,5.930405e-01,False
2,ENSG00000273203,3104,4.205864e-10,22_17581854_G_A,True,1.305500e-06,True,3.027779e-06,True
3,ENSG00000273442,3174,1.606470e-04,22_17826030_C_T,True,5.098936e-01,False,5.857847e-01,False
4,ENSG00000177663,3364,8.165293e-05,22_17596322_A_G,True,2.746804e-01,False,3.309564e-01,False
...,...,...,...,...,...,...,...,...,...
489,ENSG00000008735,8142,1.206161e-46,22_17222894_A_T,True,9.820567e-43,True,1.732629e-41,True
490,ENSG00000100299,7995,1.799452e-28,22_19276864_C_T,True,1.438662e-24,True,1.110467e-23,True
491,ENSG00000225929,7208,9.834261e-36,22_17948957_C_G,True,7.088536e-32,True,8.337468e-31,True
492,ENSG00000100312,7194,1.229523e-29,22_18918008_G_C,True,8.845186e-26,True,7.405969e-25,True


## TODOs

- [x] Run on all genes on Chromosome 22
- [x] For each gene store: minimum p value, number of variants tested, id of minimum pv variant, gene name
- [x] Bonferroni correction per hit: pv_gene = pv * num_cis_variants, np clip to (0,1)
- [ ] subset to bonferroni sginifcant hits (pv_gene < 0.05)
- [x] benjamini hochberg across tests -> qv
- [x] report # of qv < 0.05
- [ ] check how many hits you have compared to OneK1K
- [x] add gwas to tools
- [ ] Figure out how to render several notebooks
- [ ] Stretch goal: all cell types